# Orthorectification and feature matching


In [1]:
import configparser
import os
import pickle
import sys
from scipy.spatial.transform import Rotation as RotLib
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyvista as pv
from scipy.interpolate import interp1d


module_path = 'C:/Users/haavasl/VSCodeProjects/hyperspectral_toolchain/src/'
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Local resources:
from scripts.geometry import CameraGeometry, FeatureCalibrationObject, CalibHSI
from scripts.gis_tools import GeoSpatialAbstractionHSI
from lib.parsing_utils import Hyperspectral
from scripts.modulate_config import prepend_data_dir_to_relative_paths

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## We use the previously defined config file, and orthorectify the data

In [2]:
DATA_DIR = 'D:/HyperspectralDataAll/HI/2022-08-31-060000-Remoy-Specim/'
config_file_path = DATA_DIR + 'configuration.ini'
# Set the data directory for the mission (locally where the data is stored)
prepend_data_dir_to_relative_paths(config_path=config_file_path, DATA_DIR=DATA_DIR)


# Only resample RGB? This option is quicker, if refinements to the georeferencing is to be made
resample_rgb_only = False

# Chunk size in gigabytes of the resampled data
chunk_size_GB = 2


In [17]:
import numpy as np
%matplotlib qt
import spectral as sp
dir = 'D:/HyperspectralDataAll/HI/2022-08-31-060000-Remoy-Specim/Output/GIS/HSIDatacubes/'
spy_image = sp.io.envi.open(dir + '2022-08-31-060000-Remoy-Specim_transectnr_0_chunknr_0_north_east.hdr' , image = dir + '2022-08-31-060000-Remoy-Specim_transectnr_0_chunknr_0_north_east.bip')
print(spy_image.metadata)
spy_image.bands.centers = np.array(spy_image.metadata['wavelengths']).astype(float)
print(spy_image.bands.centers)
sp.imshow(spy_image)



{'description': 'Radiance converted, georeferenced data', 'samples': '2095', 'lines': '1977', 'bands': '224', 'header offset': '0', 'file type': 'ENVI Standard', 'data type': '4', 'interleave': 'bsq', 'sensor type': 'Specim FX10', 'byte order': '0', 'map info': ['UTM', '1', '1', '327712.387641734', '6919282.76670769', '0.1', '0.1', '32', 'North'], 'coordinate system string': ['PROJCS["ETRS_1989_UTM_Zone_32N"', 'GEOGCS["GCS_ETRS_1989"', 'DATUM["D_ETRS_1989"', 'SPHEROID["GRS_1980"', '6378137.0', '298.257222101]]', 'PRIMEM["Greenwich"', '0.0]', 'UNIT["Degree"', '0.0174532925199433]]', 'PROJECTION["Transverse_Mercator"]', 'PARAMETER["False_Easting"', '500000.0]', 'PARAMETER["False_Northing"', '0.0]', 'PARAMETER["Central_Meridian"', '9.0]', 'PARAMETER["Scale_Factor"', '0.9996]', 'PARAMETER["Latitude_Of_Origin"', '0.0]', 'UNIT["Meter"', '1.0]]'], 'data ignore value': '-9999', 'fwhm': ['5.21', '5.2', '5.18', '5.17', '5.16', '5.15', '5.14', '5.14', '5.13', '5.13', '5.13', '5.15', '5.15', '5.15

c:\Users\haavasl\AppData\Local\miniconda3\envs\specim\lib\site-packages\spectral\graphics\spypylab.py:796: UserWarning: Failed to create RectangleSelector object. Interactive pixel class labeling will be unavailable.
  warnings.warn(msg)


ImageView object:
  Display bands       :  [73, 50, 24]
  Interpolation       :  <default>
  RGB data limits     :
    R: [-9999.0, 9299.19140625]
    G: [-9999.0, 9389.193359375]
    B: [-9999.0, 8827.4384765625]

In [3]:
config = configparser.ConfigParser()
config.read(config_file_path)

# Paths to 3D mesh ply file 
path_mesh = config['Absolute Paths']['modelPath']

# Directory of H5 files
dir_r = config['Absolute Paths']['h5Dir']

# The path to the XML file
hsi_cal_xml = config['Absolute Paths']['HSICalibFile']

print('Orthorectifying Images')

for filename in sorted(os.listdir(dir_r)):
    if filename.endswith('h5') or filename.endswith('hdf'):
        # Path to hierarchical file
        path_hdf = dir_r + filename

        # Read h5 file
        hyp = Hyperspectral(path_hdf, config)

        point_cloud_ecef = hyp.points_global

        # Generates an object for dealing with GIS operations
        gisHSI = GeoSpatialAbstractionHSI(point_cloud=point_cloud_ecef, 
                                            transect_string=filename.split('.')[0],
                                            config=config)

        gisHSI.transform_geocentric_to_projected()

        gisHSI.footprint_to_shape_file()


        # The step for which the datacube is resampled. The GLT should be written to h5 file
        gisHSI.resample_datacube(hyp, rgb_composite_only=resample_rgb_only)

        # The ancilliary data (natural to manipulate with DEMs, annotated classes, water masks etc)
        gisHSI.resample_ancillary()

Orthorectifying Images


TypeError: resample_datacube() got an unexpected keyword argument 'chunk_size_GB'